##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# What's new in Gemini-1.5-pro-002 and Gemini-1.5-flash-002

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/New_in_002.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook explores the new options added with the 002 versions of the 1.5 series models:

* Candidate count
* Presence and frequency penalties
* Response logprobs

## Setup

Install a compatible version of the SDK:

In [2]:
%pip install -q "google-genai>=1.0.0"

import the package and give it your API-key

In [3]:
import time
from google import genai
from google.genai import types

In [4]:
from google.colab import userdata
client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

Import other packages.

In [5]:
from IPython.display import display, Markdown, HTML

Check available models that support the new features. You will use `gemini-2.0-flash-001` in this tutorial.

In [6]:
model_name = "gemini-2.0-flash-001"
test_prompt="Why don't people have tails"

## Quick refresher on `generation_config` [Optional]

In [10]:
response = client.models.generate_content(model="gemini-2.0-flash-001", contents='hello', config = types.GenerateContentConfig(temperature=1.0, max_output_tokens=5))

In [7]:
for model in client.models.list():
  print(model)
  print("-" * 20)

name='models/embedding-gecko-001' display_name='Embedding Gecko' description='Obtain a distributed representation of a text.' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo() input_token_limit=1024 output_token_limit=1 supported_actions=['embedText', 'countTextTokens'] default_checkpoint_id=None checkpoints=None
--------------------
name='models/gemini-2.5-pro-preview-03-25' display_name='Gemini 2.5 Pro Preview 03-25' description='Gemini 2.5 Pro Preview 03-25' version='2.5-preview-03-25' endpoints=None labels=None tuned_model_info=TunedModelInfo() input_token_limit=1048576 output_token_limit=65536 supported_actions=['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent'] default_checkpoint_id=None checkpoints=None
--------------------
name='models/gemini-2.5-flash-preview-05-20' display_name='Gemini 2.5 Flash Preview 05-20' description='Preview release (April 17th, 2025) of Gemini 2.5 Flash' version='2.5-preview-05-20' endpoints=None l

In [8]:
for model in client.models.list():
  print(f"Model name: {model.name}")
  print(f"Supported tasks: {model.supported_actions}")
  print("-" * 20)

Model name: models/embedding-gecko-001
Supported tasks: ['embedText', 'countTextTokens']
--------------------
Model name: models/gemini-2.5-pro-preview-03-25
Supported tasks: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
--------------------
Model name: models/gemini-2.5-flash-preview-05-20
Supported tasks: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
--------------------
Model name: models/gemini-2.5-flash
Supported tasks: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
--------------------
Model name: models/gemini-2.5-flash-lite-preview-06-17
Supported tasks: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
--------------------
Model name: models/gemini-2.5-pro-preview-05-06
Supported tasks: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
--------------------
Model name: models/gemini-2.5-pro-preview-06-05
Suppo

Note:

* Each `generate_content` request is sent with a `config` (`chat.send_message` uses `generate_content`).
* You can set the `config` by passing it in the arguments to `generate_content` (or `chat.send_message`).
* You can pass the `config` as either a Python `dict`, or a `types.GenerateContentConfig`.
* If you're ever unsure about the parameters of `config` check `types.GenerateContentConfig`.

## Candidate count

With models like `gemini-2.0-flash-001`, you can now use `candidate_count > 1`.

In [11]:
config = dict(candidate_count=2)

In [12]:
response = client.models.generate_content(model="gemini-2.0-flash-001", contents=test_prompt, config=config)

But note that the `.text` quick-accessor only works for the simple 1-candidate case.

In [13]:
try:
  response.text # Fails with multiple candidates, sorry!
except ValueError as e:
  print(e)

With multiple candidates you have to handle the list of candidates yourself:

In [14]:
for candidate in response.candidates:
  display(Markdown(candidate.content.parts[0].text))
  display(Markdown("-------------"))


Humans don't have tails because during our evolutionary history, we lost the need for them. Here's a breakdown:

*   **Our Ancestors Had Tails:** Our distant ancestors, like many other mammals, possessed tails that served various purposes, including balance, communication, and gripping.

*   **Shift to Bipedalism:** As our ancestors began to walk upright (bipedalism), the function of the tail for balance became less crucial. Our bodies gradually adapted, and the tail became less useful.

*   **Genetic Mutations:** Over generations, genetic mutations occurred that led to the shortening and eventual loss of the tail. These mutations were not necessarily harmful and, in fact, may have been advantageous in some way (e.g., reduced energy expenditure, improved balance with upright posture).

*   **Vestigial Structure:** What remains of our tail is the coccyx (tailbone) at the base of our spine. This is considered a vestigial structure, meaning it's a remnant of a feature that served a purpose in our ancestors but is no longer functional in the same way. The coccyx still serves as an attachment point for muscles and ligaments, but it's not a fully developed, external tail.

In essence, the loss of the tail in humans is a result of evolutionary adaptation to a changing lifestyle.


-------------

Humans don't have visible tails because of evolutionary changes that occurred millions of years ago. Here's a breakdown of the reasons:

*   **Our ancestors lost the need for a tail:** Early primates used tails for balance and locomotion, particularly when moving through trees. As our ancestors transitioned to walking upright on the ground, the tail became less important for balance. Other adaptations, such as changes in our inner ear and leg structure, compensated for the loss of the tail.

*   **Gene inactivation:** A specific gene, known as TBXT, plays a crucial role in tail development in vertebrates. Mutations in this gene, particularly the inactivation of a regulatory element, have been linked to tail loss in apes and humans. This inactivation prevents the tail from fully forming during embryonic development.

*   **Selective advantage:** While the exact selective pressures that led to tail loss are still debated, it's believed that having a shorter or no tail might have offered some advantages to early humans. One theory suggests that a tail could have become cumbersome or even a hindrance as our ancestors adapted to bipedalism.

In summary, the loss of our tail is a result of evolutionary processes that include changes in gene function and adaptation to a new lifestyle on the ground.

-------------

The response contains multiple full `Candidate` objects.

In [15]:
response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      avg_logprobs=-0.36429186834805255,
      content=Content(
        parts=[
          Part(
            text="""Humans don't have tails because during our evolutionary history, we lost the need for them. Here's a breakdown:

*   **Our Ancestors Had Tails:** Our distant ancestors, like many other mammals, possessed tails that served various purposes, including balance, communication, and gripping.

*   **Shift to Bipedalism:** As our ancestors began to walk upright (bipedalism), the function of the tail for balance became less crucial. Our bodies gradually adapted, and the tail became less useful.

*   **Genetic Mutations:** Over generations, genetic mutations occurred that led to the shortening and eventual loss of the tail. These mutations were not necessarily harmful and, in fact, may have been advantageous in some way (e.g., reduced energy expenditure, improved balance with upright po

## Penalties

The `1.5` models expose `penalty` arguments that let you affect the statistics of output tokens.

### Presence penalty

The `presence_penalty` penalizes tokens that have already been used in the output, so it induces variety in the model's output. This is detectible if you count the unique words in the output.

Here's a function to run a prompt a few times and report the fraction of unique words (words don't map perfectly to tokens but it's a simple way to see the effect).

In [16]:
from statistics import mean

In [17]:
def unique_words(prompt, config, N=10, model_name="gemini-2.0-flash-001"):
  responses = []
  vocab_fractions = []
  for n in range(N):
    # Add a delay to avoid hitting rate limits
    time.sleep(1)
    response = client.models.generate_content(model=model_name, contents=prompt, config=config)
    responses.append(response)

    words = response.text.lower().split()
    score = len(set(words))/len(words)
    print(score)
    vocab_fractions.append(score)

  return vocab_fractions

In [18]:
prompt='Tell me a story'

In [19]:
# baseline
v = unique_words(prompt, config={}, model_name="gemini-2.0-flash-001")

0.5057179161372299
0.5364667747163695
0.5081967213114754
0.5427728613569321
0.5026525198938993
0.5457685664939551
0.520863309352518
0.5485714285714286
0.5518394648829431
0.5559055118110237


In [20]:
mean(v)

0.5318755074527775

In [21]:
# the penalty encourages diversity in the oputput tokens.
v = unique_words(prompt, config=dict(presence_penalty=1.999), model_name="gemini-2.0-flash-001")

0.5680379746835443
0.5474452554744526
0.48531289910600256
0.5317220543806647
0.5244444444444445
0.5608974358974359
0.5678776290630975
0.4830917874396135
0.5462962962962963
0.5225505443234837


In [22]:
mean(v)

0.5337676321109035

In [23]:
# a negative penalty discourages diversity in the output tokens.
v = unique_words(prompt, config=dict(presence_penalty=-1.999), model_name="gemini-2.0-flash-001")

0.5797546012269938
0.5617792421746294
0.5495356037151703
0.5064562410329986
0.5529411764705883
0.4880239520958084
0.5755395683453237
0.5453125
0.5809199318568995
0.556782334384858


In [24]:
mean(v)

0.549704515130327

The `presence_penalty` has a small effect on the vocabulary statistics.

### Frequency Penalty

Frequency penalty is similar to the `presence_penalty` but  the penalty is multiplied by the number of times a token is used. This effect is much stronger than the `presence_penalty`.

The easiest way to see that it works is to ask the model to do something repetitive. The model has to get creative while trying to complete the task.

In [25]:
response = client.models.generate_content(model="gemini-2.0-flash-001", contents='please repeat "Cat" 50 times, 10 per line',
                                  config=dict(frequency_penalty=1.999))

In [26]:
print(response.text)

Cat Cat Cat Cat Cat Cat Cat Cat Cat Cat
Cat Cat Cat Cat Cat Cat Cat Cat Cat Cat
Cat Cat Cat Cat Cat Cat Cat Cat Cat Cat
Cat Cat Cat Cat Cat Cat Cat Cat Cat Cat
Cat Cat Cat Cat Cat Cat Cat Cat Cat Cat



Since the frequency penalty accumulates with usage, it can have a much stronger effect on the output compared to the presence penalty.

> Caution: Be careful with negative frequency penalties: A negative penalty makes a token more likely the more it's used. This positive feedback quickly leads the model to just repeat a common token until it hits the `max_output_tokens` limit (once it starts the model can't produce the `<STOP>` token).

In [27]:
response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents=prompt,
    config=types.GenerateContentConfig(
        max_output_tokens=400,
        frequency_penalty=-2.0))

In [28]:
Markdown(response.text)  # the, the, the, ...

The wind howled a mournful song through the skeletal branches of the ancient oak, a sound Elara had grown accustomed to. She lived in a cabin nestled so deep within the Whispering Woods that the outside world felt like a faded dream. Her only companion was Flicker, a scruffy, one-eared fox with fur the color of dried leaves.

Elara wasn't lonely, not exactly. The woods whispered stories of their own, tales woven from the rustling leaves, the chirping crickets, and the babbling brook that snaked behind her cabin. She was a weaver of these stories, her fingers nimble as she coaxed the tales from the threads of her loom.

But lately, the stories had grown dim. The vibrant colors of her yarns seemed muted, mirroring a hollowness in her own heart. She missed the vibrant hues that had once danced in her creations, the spark of magic that had brought her tapestries to life.

One particularly bleak morning, Elara stumbled upon a strange sight. Nestled amongst the gnarled roots of the oak was a small, wooden box, intricately carved with symbols she didn't recognize. Curiosity tugged at her, a feeling she hadn't felt in months.

With trembling fingers, she opened the box. Inside, nestled on a bed of moss, lay a single, iridescent feather. It shimmered with all the colors Elara had lost – emerald green, sapphire blue, ruby red, and everything in between. It pulsed with a faint, warm light.

Flicker, usually wary of new things, sniffed the feather cautiously, then nudged it with his nose, a soft whimper escaping his throat. Elara felt a surge of something akin to hope.

Hesitantly, she picked up the feather. As her fingers brushed against its silken surface, a jolt of energy surged through her. Visions flooded her mind: soaring birds with plumage like rainbows,

In [29]:
response.candidates[0].finish_reason

<FinishReason.MAX_TOKENS: 'MAX_TOKENS'>